In [7]:
import torch as th
import torch.nn as nn
import numpy as np
import scipy.io as sp
from functools import partial
from pyDOE import lhs

iter = 0
MU = 0.01
RHO = 1.0

In [4]:
def set_seed(seed: int = 42):
    '''
    Seeding the random variables for reproducibility
    ''' 
    th.manual_seed(seed)
    th.cuda.manual_seed(seed)
    th.cuda.manual_seed_all(seed)
    th.backends.cudnn.deterministic = True
    th.backends.cudnn.benchmark = False

In [5]:
class NSNN(nn.Module):
    def __init__(self,):
        # Input layer
        super(NSNN, self).__init__()
        self.linear_in = nn.Linear(2, 100)
        # Output layer
        self.linear_out = nn.Linear(100, 2)
        # Hidden Layers
        self.layers = nn.ModuleList(
            [ nn.Linear(100, 100) for i in range(3) ]
        )
        # Activation function
        self.act = nn.Tanh()

    def forward(self, x: th.Tensor) -> th.Tensor:
        x = self.linear_in(x)
        for layer in self.layers:
            x = self.act(layer(x))
        x = self.linear_out(x)
        return x

In [6]:
def derivative(dy: th.Tensor, x: th.Tensor, order: int = 1) -> th.Tensor:
    """
    This function calculates the derivative of the model at x_f
    """
    for i in range(order):
        dy = th.autograd.grad(
            dy, x, grad_outputs = th.ones_like(dy), create_graph=True, retain_graph=True
        )[0]
    return dy

In [ ]:
def f(model, x_f, y_f, t_f):
    """
    This function evaluates the PDE at collocation points.
    """
    hidden = model(th.stack((x_f, y_f, t_f), axis = 1))
    psi = hidden[:, 0]
    # predictions
    p = hidden[:, 1]
    u = derivative(psi, y_f, order=1)
    v = -derivative(psi, x_f, order=1)
    
    # first order derivatives
    u_x = derivative(u, x_f, order=1)
    u_y = derivative(u, y_f, order=1)
    u_t = derivative(u, t_f, order=1)
    
    v_x = derivative(v, x_f, order=1)
    v_y = derivative(v, y_f, order=1)
    v_t = derivative(v, t_f, order=1)
    
    p_x = derivative(p, x_f, order=1)
    p_y = derivative(p, y_f, order=1)
    
    # second order derivatives
    u_xx = derivative(u, x_f, order=2)
    u_yy = derivative(u, y_f, order=2)
    
    v_xx = derivative(v, x_f, order=2)
    v_yy = derivative(v, y_f, order=2)
    
    # PDEs
    f_u =  u_t + u*u_x + v*u_y + p_x - MU*(u_xx + u_yy) / RHO
    f_v =  v_t + u*v_x + v*v_y + p_y - MU*(v_xx + v_yy) / RHO
    f_uv = u_x + v_y

    return f_u, f_v, f_uv

def mse_f(model, x_f, y_f, t_f):
    """
    This function calculates the MSE for the PDE.
    """
    f_u, f_v, f_uv = f(model, x_f, y_f, t_f)
    return (f_u**2 + f_v**2 + f_uv**2).mean()

def mse_0(model, x_0, y_0):
    """
    This function calculates the MSE for the initial condition.
    """
    # creating a t_0 variable to be the same shape as the x_0 but with zero values
    t_0 = th.zeros_like(x_0)
    hidden = model(th.stack((x_0, y_0, t_0), axis = 1))
    # extracting the u and v values from the model output
    psi = hidden[:, 0]
    
    # predictions
    p = hidden[:, 1]
    u = derivative(psi, y_0, order=1)
    v = -derivative(psi, x_0, order=1)
    return ((h)**2+(h_v-v_0)**2).mean()